<a href="https://colab.research.google.com/github/jensengroup/protonator/blob/main/protonator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rdkit-pypi
!pip install mols2grid

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 22.7 MB 1.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wget https://raw.githubusercontent.com/czodrowskilab/Machine-learning-meets-pKa/master/datasets/combined_training_datasets_unique.sdf

--2022-06-03 11:29:19--  https://raw.githubusercontent.com/czodrowskilab/Machine-learning-meets-pKa/master/datasets/combined_training_datasets_unique.sdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10219941 (9.7M) [text/plain]
Saving to: ‘combined_training_datasets_unique.sdf’

combined_training_d 100%[===================>]   9.75M  --.-KB/s    in 0.1s    

2022-06-03 11:29:19 (82.0 MB/s) - ‘combined_training_datasets_unique.sdf’ saved [10219941/10219941]



In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from collections import defaultdict
from rdkit.Chem import rdFMCS
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors, rdMolDescriptors

from rdkit.Chem import rdmolops
from rdkit.Chem.MolStandardize import rdMolStandardize   

import mols2grid
import pandas as pd
import os

In [ ]:
#From https://www.rdkit.org/docs/Cookbook.html#neutralizing-molecules
def uncharge(mol):
  pattern = Chem.MolFromSmarts("[+1!h0!$([*]~[-1,-2,-3,-4]),-1!$([*]~[+1,+2,+3,+4])]")
  at_matches = mol.GetSubstructMatches(pattern)
  at_matches_list = [y[0] for y in at_matches]
  if len(at_matches_list) > 0:
    for at_idx in at_matches_list:
      atom = mol.GetAtomWithIdx(at_idx)
      chg = atom.GetFormalCharge()
      hcount = atom.GetTotalNumHs()
      atom.SetFormalCharge(0)
      atom.SetNumExplicitHs(hcount - chg)
      atom.UpdatePropertyCache()
  return mol

In [ ]:
def run_rxn(reactant, smarts): 
  ps = True
  while ps:
    rxn = AllChem.ReactionFromSmarts(smarts)
    ps = rxn.RunReactants((reactant,))
    if ps:
      reactant = ps[0][0] 
      Chem.SanitizeMol(reactant)

  return reactant

def protonator(m):
  rxns = ['[NX3;H2;!$(NC=[O,S]);!$(Na);!$(*[N,S,O,P]);!$(NC=[N;H0]);!$(NCC(F)F);!$(NCC#[C,N]);!$(NCC[ND2]);!$(NC=N):1]>>[NH3+:1]', #Primary aliphatic amines
          '[NX3;H1;!$(NC=[O,S]);!$(Na);!$(*[N,S,O,P]);!$(NC=[N;H0]);!$(NCC(F)F);!$(NCC#[C,N]);!$([N;R][C;R][O,S,P;R]);!$([N;R][C;R][C;R][O,S,P;R]);!$(NC[N+]);!$(NCC[N+]);!$(NC=N):1]>>[NH2+:1]', #Secondary aliphatic amines
          '[NX3;H0;!$(NC=[O,S]);!$(Na);!$(*[N,S,O,P]);!$(NC=[N;H0]);!$(NCC(F)F);!$(NCC#[C,N]);!$([N;R][C;R][O,S,P;R]);!$([N;R][C;R][C;R][O,S,P;R]);!$(NC[N+]);!$(NCC[N+]);!$(NC=N):1]>>[NH+:1]', #Tertiary aliphatic amines
          '[NX2;H0;!$(NC=[O,S]);!$(Na);!$(*=[N,S,O,P]);!$(*=[C;R][S;R]):1]>>[NH+:1]', #sp2 N
          '[NX2;H1;!$(NC=[O,S]);!$(Na);!$(*=[N,S,O,P]);!$(*=[C;R][S;R]):1]>>[NH2+:1]', #sp2 N
          #'[nX3;H1;$(*(n)n):1]>>[n-;H0:1]', #1,2,3-triazole
          '[nX3;H1;$(*(n)nn):1]>>[n-;H0:1]', #tetrazole
          '[O;H1;$(O[C,S]=O):1]>>[O-;H0:1]', #carboxylic and sulfinic acid
          '[O;H1;$(O[C,c;R]=[C,c;R][C,c]=O):1]>>[O-;H0:1]','[O;H1;$(Occc=O):1]>>[O-;H0:1]', #Phenol near C=O
          '[O;H1;$(Oaaa[nX3+]):1]>>[O-;H0:1]','[O;H1;$(Oaa[nX3+]):1]>>[O-;H0:1]','[O;H1;$(Oa[nX3+]):1]>>[O-;H0:1]','[O;H1;$(O[nX3+]):1]>>[O-;H0:1]', #Phenol near pyridinium
          '[O;H1;$(Ocnnn):1]>>[O-;H0:1]', '[O;H1;$(Onnn):1]>>[O-;H0:1]', #Phenol on 1,2,3-triazole
          '[O;H1;$(Oc(c[F,Cl,Br,I])c[F,Cl,Br,I]):1]>>[O-;H0:1]', #Phenol flanked by halides
          '[O;H1;$(OP(=[O,S])):1]>>[O-;H0:1]'] #Phosphate O

  for rxn in rxns:
    molb = run_rxn(m, rxn)
    if molb:
      m = molb

  return m

In [ ]:
suppl = Chem.SDMolSupplier('combined_training_datasets_unique.sdf')
mols = [m for m in suppl]
len(mols)

5994

In [ ]:
df = pd.DataFrame({'smiles': [Chem.MolToSmiles(m) for m in mols]})
df['id'] = [i for i in range(len(mols))]
df['pKa'] = [float(m.GetProp('pKa')) for m in mols]
df['Marvin'] = [float(m.GetProp('marvin_pKa')) for m in mols]
df['n charge'] = [rdmolops.GetFormalCharge(uncharge(m)) for m in mols]
df['pKa type'] = [m.GetProp('marvin_pKa_type') for m in mols]
df['acidic atom'] = [m.GetAtomWithIdx(int(m.GetProp('marvin_atom'))).GetSymbol() for m in mols]

In [ ]:
mols2grid.display(df.head(500),
                  fixed_bond_length=25,
                  smiles_col = 'smiles',
                  subset=["img", 'id', 'pKa', 'Marvin'],
                  n_rows = 5,
                  n_cols = 8,
                  )

In [ ]:
pred_charge = []
for i,m in enumerate(mols):
  try:
    new_m = protonator(uncharge(m))
    pred_charge.append(rdmolops.GetFormalCharge((new_m)))
  except:
    print(i)
    break

df['pred_charge'] = pred_charge

acidic = df.loc[df['pKa type'] == 'acidic']
basic = df.loc[df['pKa type'] == 'basic']

print(acidic.shape,basic.shape)

(2398, 8) (3596, 8)


In [ ]:
#wrong predictions for O-acids with pKa values < 6.0
acidic_w = acidic.loc[(acidic['acidic atom'] == 'O') & (acidic['pred_charge'] > -1 + acidic['n charge']) & (acidic['pKa'] < 6.0)]

acidic_w.shape

(16, 8)

In [ ]:
mols2grid.display(acidic_w,
                  fixed_bond_length=25,
                  smiles_col = 'smiles',
                  subset=["img", 'id', 'pKa', 'Marvin'],
                  n_rows = 7,
                  n_cols = 8,
                  )

In [ ]:
#wrong predictions for N-bases with pKa values < 6.0
basic_w = basic.loc[(basic['acidic atom'] == 'N') & (basic['pred_charge'] > 0 + basic['n charge']) & (basic['pKa'] < 6.0)]

basic_w.shape

(82, 8)

In [ ]:
mols2grid.display(basic_w,
                  fixed_bond_length=25,
                  smiles_col = 'smiles',
                  subset=["img", 'id', 'pKa', 'Marvin'],
                  n_rows = 7,
                  n_cols = 8,
                  )

In [ ]:
smiles = ['NCNC', 'NCN(C)C', 'CNCN(C)C', 'CNCNC', 'CN(C)CN(C)C',
          'NCCNC', 'NCCN(C)C', 'CNCCN(C)C', 'CNCCNC', 'CN(C)CCN(C)C',
          'NCCNCNC', 'CN1CCNCC1', 'CN1CCN(CCN)CC1', 'CCN1CC(N)C1', 'CCN1CC(CN)C1',
          'NC1CCCNC1', 'N=C(N)NC']
mols = [Chem.MolFromSmiles(s) for s in smiles]

p_mols = [protonator(m) for m in mols]

df = pd.DataFrame({'mols': p_mols})
mols2grid.display(df,
                  fixed_bond_length=25,
                  mol_col = 'mols',
                  subset=["img"],
                  n_rows = 7,
                  n_cols = 5,
                  )